In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
import itertools
import random 
import sklearn


from sklearn import linear_model, tree, ensemble


In [ ]:
df = pd.read_csv('train.csv')
df.tail()

In [ ]:
#droppare dati vuoti df.dropna() ma non ce ne sono
df.dropna()
df.index.size

In [ ]:
df['churn'].hist();

In [ ]:
df[df['churn'] == ('yes')]['churn'].count()

In [ ]:
df[df['churn'] == ('no')]['churn'].count()

In [ ]:
df['number_customer_service_calls'].hist();

In [ ]:
df['number_vmail_messages'].hist();

In [ ]:
cyn = df[df['churn'] == 'yes']
cyn.groupby('state').size().plot(kind = 'bar', figsize=(20,10));

In [ ]:
(df.groupby(['state','churn'])['churn'].size() / df.groupby(['state']).size()).plot(kind = 'bar', figsize=(20,10));

In [ ]:
plt.figure(figsize=(12,10))
cor = df.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show();

In [ ]:
df[['total_day_charge', 'total_eve_charge', 'total_night_charge']].sum().plot(kind='bar');

In [ ]:
df1 = df.drop(['state', 'area_code',
               'total_day_calls','total_eve_calls','total_night_calls',
               'total_day_minutes','total_eve_minutes','total_night_minutes',
               'total_day_charge','total_eve_charge','total_night_charge',
               'total_intl_calls','total_intl_minutes','total_intl_charge'], axis=1,)
df1

In [ ]:
df1['total_call'] = df['total_day_calls'] + df['total_eve_calls'] + df['total_night_calls']
df1['total_minutes'] = df['total_day_minutes'] + df['total_eve_minutes'] + df['total_night_minutes']
df1['total_charge'] = df['total_day_charge'] + df['total_eve_charge'] + df['total_night_charge']
df1['total_intl'] = df['total_intl_calls'] + df['total_intl_minutes'] + df['total_intl_charge']
df1

In [ ]:
plt.figure(figsize=(12,10))
cor = df1.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Provo a trovare score DT e poi CV per uno score migliore

In [ ]:
#encoding variabili categoriche

from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()
df1["international_plan"] = ord_enc.fit_transform(df1[["international_plan"]])
df1["voice_mail_plan"] = ord_enc.fit_transform(df1[["voice_mail_plan"]])
df1["churn"] = ord_enc.fit_transform(df1[["churn"]])

In [ ]:
#selezione colonne per train modello

X = df1[['international_plan','voice_mail_plan','number_customer_service_calls', 'account_length']] #Two-dimensional (pd DataFrame)
y = df1['churn'] #One-dimensional (pd Series)

In [ ]:
#train_test_split() di sklearn.model_selection splitta il df in due parti (in questo caso train e test da 80%20%)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,
                                                    random_state=1,
                                                    shuffle=True)

In [ ]:
#modello decision tree, fit + score

from sklearn.tree import DecisionTreeClassifier

#modello 
dtc1 = DecisionTreeClassifier(max_depth=3,
                              random_state=1)

# fit()
dtc1.fit(X_train, y_train)

#.score() del tree ritorna 'the mean accuracy on the given test data and labels'
dtc1.score(X_train, y_train)

In [ ]:
#grafico del tree 

plt.figure(figsize=(15,10))
tree.plot_tree(dtc1);

In [ ]:
#creazione pdf con grafico del tree

from sklearn.tree import export_graphviz

dot_data = export_graphviz(dtc1,
                           feature_names=['international_plan','voice_mail_plan','number_customer_service_calls', 'account_length'],
                           filled=True,
                           rounded=True)

graph = graphviz.Source(dot_data)
graph.render("tree")

In [ ]:
#GridSearch() di sklearn.model_selection aiuta a capire gli iperparametri

from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(dtc1,
                  param_grid = {'max_depth': range(1, 10),                #prova con max_depth dell'albero da 1 a 10
                                'min_samples_split': range(10, 60, 10)},  #divide il df in parti da 10 a 60 con intervalli di 10
                  cv=5,
                  n_jobs=1,
                  scoring='accuracy') # come scoring uso errore 'accuracy' che è relativo ad un algoritmo classificatore'

gs.fit(X_train, y_train)

print(gs.best_params_)
print(gs.best_score_)

In [ ]:
#riassegno l'iperparametro 'max_depth' dopo averlo migliorato con GridSearchCV()

dtc1 =  DecisionTreeClassifier(max_depth=8,
                               random_state=1)

In [ ]:
#KFold() per splittare il df in 5 e usare 4 parti come training e 1 come test per 5 volte
#in modo da usare ogni parte come test almeno 1 volta

from sklearn.model_selection import KFold

kf =KFold(n_splits=5,
          shuffle=True,
          random_state=42)

#split()  method generate indices to split data into training and test set

cnt = 1
for train_index, test_index in kf.split(X_train, y_train):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt += 1

In [ ]:
#cross validation del decision tree sul df

from sklearn.model_selection import cross_val_score

cv_score_dtc = cross_val_score(dtc1,
                               X_train,
                               y_train,
                               cv = kf,
                               scoring="accuracy")


print(f'Scores for each fold are: {cv_score_dtc}')
print(f'Average score: {"{:.2f}".format(cv_score_dtc.mean())}')

In [ ]:
#Provo a trovare score KNN e poi CV per uno score migliore

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knc = KNeighborsClassifier(n_neighbors = 3)

#fit()
knc.fit(X_train, y_train)

#.score() ritorna 'the mean accuracy on the given test data and labels'
knc.score(X_train, y_train) 

In [ ]:
#valutazione miglior iperparametro n_neighbors

IP = []
for i in range(1, 50):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    s = knn.score(X_train, y_train)
    IP.append({'best n' : i,
               'score': s})

IP = pd.DataFrame(IP)
IP.sort_values('score').tail(1)  #valore suggerito di n_neighbors

In [ ]:
#riassegno l'iperparametro n_neighbors dopo averlo migliorato il loop 

knc = KNeighborsClassifier(n_neighbors = 1)

In [ ]:
#cross validation del knn sul df
cv_score_knn = cross_val_score(knc,
                               X_train,
                               y_train,
                               cv = kf,
                               scoring="accuracy")


print(f'Scores for each fold are: {cv_score_knn}')
print(f'Average score: {"{:.2f}".format(cv_score_knn.mean())}')

In [ ]:
#faccio il fit dell'intero df usando il DECISION TREE CLASSIFIER come algoritmo
#dato che la media delle 'accuracy' del cross validation è più alta di quella trovata con il KNN

In [ ]:
#X_train, y_train per predire X_test

dtc_finale = DecisionTreeClassifier(max_depth=8,
                                    random_state=1)
dtc_finale.fit(X_train, y_train)

dtc_finale.predict(X_test)

In [ ]:
#X, y per predire il file con nuovi dati

dtc_finale2 = DecisionTreeClassifier(max_depth=8,
                                    random_state=1)

dtc_finale2.fit(X, y)

test = pd.read_csv('test.csv')
test.head()

In [ ]:
#riassegnazione nuove variabili
ord_enc = OrdinalEncoder()
test["international_plan"] = ord_enc.fit_transform(test[["international_plan"]])
test["voice_mail_plan"] = ord_enc.fit_transform(test[["voice_mail_plan"]])

Xf = test[['international_plan','voice_mail_plan','number_customer_service_calls','account_length']]


In [ ]:
dtc_finale2.predict(Xf)